In [2]:
## Kevin Wang RJ Lombardi Alex Wan
## Paragraphs are uploaded on CMS

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt


In [3]:
hp = pd.read_csv("train.csv")
hp1 = pd.read_csv("train.csv")

In [23]:
pd.set_option('display.max_columns', None)
target=hp['SalePrice']
target = target/75000
hp.head()
hp.corr()
target = target.astype(int)
features=hp[['OverallQual','GrLivArea','1stFlrSF']]

In [229]:
#KNN
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut 

train_features, valid_features, train_target, valid_target = train_test_split(features, target,
                                                                              test_size=0.2, random_state=353)
knn = KNeighborsClassifier(n_neighbors=2, algorithm = 'auto')
knn.fit(train_features, train_target )
predictions = knn.predict(valid_features)

kf = KFold(n_splits = 3)
split2, split1, split3 = kf.split(train_features, train_target)



In [230]:
# SVM - uses first fold
X_train_SVM = train_features.iloc[split1[0]]
X_test_SVM = train_features.iloc[split1[1]]
Y_train_SVM = train_target.iloc[split1[0]]
Y_test_SVM = train_target.iloc[split1[1]]

from sklearn import svm
SVM = svm.SVC()
SVM.fit(X_train_SVM, Y_train_SVM)
SVM_pred = SVM.predict(X_test_SVM)

SVM_score = (Y_test_SVM == SVM_pred).sum() / len(SVM_pred)
a = [word[1] for word in Counter(Y_test_SVM).most_common(1)]
baseline_acc = a[0]/len(Y_test_SVM)
print("SVM: " + str(SVM_score))
print(baseline_acc)

SVM: 0.470437017995
0.42159383033419023


In [231]:
# kNN - uses second fold
from collections import Counter
X_train_kNN = train_features.iloc[split2[0]]
X_test_kNN = train_features.iloc[split2[1]]
Y_train_kNN = train_target.iloc[split2[0]]
Y_test_kNN = train_target.iloc[split2[1]]

from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier()
kNN.fit(X_train_kNN, Y_train_kNN)
kNN_pred = kNN.predict(X_test_kNN)

kNN_score = (Y_test_kNN == kNN_pred).sum() / len(kNN_pred)
a = [word[1] for word in Counter(Y_test_kNN).most_common(1)]
baseline_acc = a[0]/len(Y_test_kNN)
print("kNN: " + str(kNN_score))
print(baseline_acc)

kNN: 0.610256410256
0.4


In [232]:

from sklearn.tree import DecisionTreeClassifier

X_train_tree = train_features.iloc[split3[0]]
X_test_tree = train_features.iloc[split3[1]]
Y_train_tree = train_target.iloc[split3[0]]
Y_test_tree = train_target.iloc[split3[1]]

tree = DecisionTreeClassifier()
tree.fit(X_train_tree, Y_train_tree)
tree_pred = tree.predict(X_test_tree)

tree_score = (Y_test_tree == tree_pred).sum() / len(tree_pred)
a = [word[1] for word in Counter(Y_test_tree).most_common(1)]
baseline_acc = a[0]/len(Y_test_tree)
print("Decision Tree: " + str(tree_score))
print(baseline_acc)

Decision Tree: 0.622107969152
0.3856041131105398


In [233]:
# stacking
# combines three folds
import numpy as np
kNN_pred = kNN_pred.reshape(-1,1)
SVM_pred = SVM_pred.reshape(-1,1)
tree_pred = tree_pred.reshape(-1,1)
pred_input = np.vstack([kNN_pred, SVM_pred, tree_pred])

from sklearn.linear_model import LogisticRegression
stacked = LogisticRegression()
stacked.fit(pred_input, train_target)

# the first layer predicts
pred_test = pd.DataFrame({'kNN': kNN.predict(valid_features), 'SVM': SVM.predict(valid_features), 'tree': tree.predict(valid_features)})
# the average of the predictions is calculated
pred_test = pred_test.assign(avg=pred_test.mean(axis=1))
stacked_pred = stacked.predict(pred_test[['avg']])

stacked_score = (valid_target == stacked_pred).sum() / len(stacked_pred)
print("Stacked: " + str(stacked_score))

Stacked: 0.650684931507
